<a href="https://colab.research.google.com/github/h1z1z1h16584/colab/blob/main/File_Downloader_with_Threading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#@title <-----<strong> Mounting Google Drive</strong>
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title <-----<strong> Download and Install Libraries</strong>
!pip install requests
!pip install tqdm
!pip install ipywidgets
from google.colab import output
from IPython.display import clear_output
output.enable_custom_widget_manager()
clear_output(wait=True)

from tqdm import tqdm
import os
import sys
from pathlib import Path
import requests
import threading
import ipywidgets as widgets
from IPython.display import display

# Define the download function
def download(url, filename=None):
    """
    Downloads a file from a given URL with a progress bar.
    """
    try:
        response = requests.head(url, allow_redirects=True)
        filesize = int(response.headers["Content-Length"])

        if filename is None:
            filename = os.path.basename(response.url)

        print(f"Downloading {filename} ({filesize} bytes)...")

        home_path = "/content/drive/MyDrive"
        sub_path = "downloads"
        os.makedirs(os.path.join(home_path, sub_path), exist_ok=True)
        dl_path = os.path.join(home_path, sub_path, filename)

        chunk_size = 1024
        with requests.get(url, stream=True, allow_redirects=True) as r, open(dl_path, "wb") as f, tqdm(
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
            total=filesize,
            file=sys.stdout,
            desc=filename
        ) as progress:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    datasize = f.write(chunk)
                    progress.update(datasize)
    except requests.exceptions.RequestException as e:
        print(f"\nError downloading {url}: {e}")
    except KeyError:
        print(f"\nError: Could not determine file size for {url}. The 'Content-Length' header was not found.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


In [43]:
#@title <-----<strong> Adding URLs and File names</strong>
urls_and_filenames = []

# Create the UI widgets
url_input = widgets.Text(description="URL:")
filename_input = widgets.Text(description="New Filename (optional):")
add_button = widgets.Button(description="Add URL")
output_area = widgets.Output()

simultaneous_download_checkbox = widgets.Checkbox(
    value=True,
    description="Simultaneous Download",
    disabled=False,
    indent=False
)
display(simultaneous_download_checkbox)

def on_add_button_clicked(b):
    with output_area:
        url = url_input.value
        filename = filename_input.value if filename_input.value else None

        if url:
            urls_and_filenames.append((url, filename))
            print(f"Added: {url} -> {filename}")
            url_input.value = ""
            filename_input.value = ""
        else:
            print("Please enter a valid URL.")

add_button.on_click(on_add_button_clicked)

display(url_input, filename_input, add_button, output_area)

# Now, you need to manually run the next cell when you are done adding URLs.
print("When you are done adding URLs, run the next cell to start the downloads.")

Checkbox(value=True, description='Simultaneous Download', indent=False)

Text(value='', description='URL:')

Text(value='', description='New Filename (optional):')

Button(description='Add URL', style=ButtonStyle())

Output()

When you are done adding URLs, run the next cell to start the downloads.


In [47]:
#@title <-----<strong> Start Downloading</strong>
print("\nStarting downloads...")
if simultaneous_download_checkbox.value:
    print("Mode: Simultaneous")
    threads = []
    for url, new_filename in urls_and_filenames:
        t = threading.Thread(target=download, args=(url, new_filename))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()
    print("\nAll simultaneous downloads finished.")
else:
    print("Mode: Sequential")
    for url, new_filename in urls_and_filenames:
        download(url, new_filename)

    print("\nAll sequential downloads finished.")


Starting downloads...
Mode: Simultaneous
68aba478f75526a5526e9e8a:   0%|          | 0.00/1.11G [00:00<?, ?B/s]
68aba478f75526a5526e9e8a:   0%|          | 94.0k/1.11G [00:00<55:47, 356kB/s]  
68aba478f75526a5526e9e8a:   0%|          | 230k/1.11G [00:00<26:51, 740kB/s] 
68aba478f75526a5526e9e8a:   0%|          | 366k/1.11G [00:00<20:47, 956kB/s]
68aba478f75526a5526e9e8a:   0%|          | 740k/1.11G [00:00<10:33, 1.88MB/s]
68aba478f75526a5526e9e8a:   0%|          | 1.49M/1.11G [00:00<05:14, 3.79MB/s]
68aba478f75526a5526e9e8a:   0%|          | 2.55M/1.11G [00:00<03:18, 6.00MB/s]
68aba478f75526a5526e9e8a:   0%|          | 5.12M/1.11G [00:00<01:36, 12.3MB/s]
68aba478f75526a5526e9e8a:   1%|          | 7.50M/1.11G [00:01<01:14, 15.9MB/s]
68aba478f75526a5526e9e8a:   1%|          | 10.1M/1.11G [00:01<01:02, 19.0MB/s]
68aba478f75526a5526e9e8a:   1%|          | 12.5M/1.11G [00:01<00:57, 20.6MB/s]
68aba478f75526a5526e9e8a:   1%|▏         | 14.9M/1.11G [00:01<00:53, 22.0MB/s]
68aba478f75526a5526e9e

KeyboardInterrupt: 